In [2]:
from pathlib import Path
from typing import Literal


def parser(file_path: str) -> tuple[list[str|dict[str, list[dict[str, list[str]]]]], list[str]]|Exception:
    file = Path(file_path)
    if not file.exists():
        return OSError(f"File does not exist: {file_path}")
    context: str = file.read_text("utf-8", errors="ignore")
    return parser1(context)

def parser1(context: str) -> tuple[list[str|dict[str, list[dict[str, list[str]]]]], list[str]]|Exception:
    
    state:Literal["Unknown","maybe_S_key", "prefix_dot", "S_key", "S_value", "maybe_S_key_or_value", "S_value_with_quote", "Note"] = "Unknown"
    data:list[str|dict[str, list[dict[str, list[str]]]]] = []
    data_raw_string:list[str] = []
    current_field:dict[str, list[dict[str, list[str]]]] = {}
    current_note:str = ""
    current_p_key: str = ""
    current_p_value: dict[str, list[str | bool | int | float]] = {}
    current_s_key: str = ""
    current_s_value: list[str | bool | int | float] = []
    temp_string:str = ""
    raw_string:str = ""

    for char in context:
        raw_string += char
        match (char,state):
            case (":","Unknown"):
                current_p_key = temp_string.split("\n")[-1]
                temp_string = ""
                state = "maybe_S_key"
            case ("/","Unknown"):
                if temp_string[-2:] == "//":
                    temp_string = ""
                    state = "Note"
            case (".","Unknown"):
                temp_string = ""
                state = "prefix_dot"

            case ("\n","Note"):
                current_note = temp_string
                data.append(current_note)
                data_raw_string.append(raw_string)


            case (".","maybe_S_key"):
                temp_string = ""
                state = "S_key"

            case ('"',"S_key"):
                current_s_key = temp_string
                temp_string = ""
                state = "S_value_with_quote"
            case ("0"|"1"|"2"|"3"|"4"|"5"|"6"|"7"|"8"|"9","S_key"):
                current_s_key = temp_string
                temp_string=char
                state = "S_value"
            case (" ","S_key"):
                current_s_key = temp_string
                temp_string = ""
                state = "Unknown"

            case (" "|"\t"|"\n","S_value"):
                current_s_value.append(temp_string)
                temp_string = ""
                state = "Unknown"

            case ('"',"S_value_with_quote"):
                current_s_value.append(temp_string)
                temp_string = ""
                state = "Unknown"

            case ('"',"Unknown"):
                temp_string = ""
                state = "S_value_with_quote"
            case _:
                raw_string += char
        temp_string += char

    return (data, data_raw_string)

parser1(open(r"D:\Program Files\MO2\Game\DarkestDunGeon\mods\The Abigail Williams Class\heroes\AbigailWilliams\AbigailWilliams.info.darkest").read())

([], [])

In [6]:
"123//"[-2:]

'//'